In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased', use_fast=True)

print(tokenizer)
# 编码试算
tokenizer.batch_encode_plus([
    'hello, everyone, today is a good day',
    'how are you , fine thank you , and you?'
])

DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


{'input_ids': [[101, 7592, 1010, 3071, 1010, 2651, 2003, 1037, 2204, 2154, 102], [101, 2129, 2024, 2017, 1010, 2986, 4067, 2017, 1010, 1998, 2017, 1029, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [5]:
from datasets import load_dataset

dataset = load_dataset(path='glue', name='cola')
dataset

Found cached dataset glue (C:/Users/SupercoldZzz/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})

In [3]:
dataset['train'][0]

{'sentence': "Our friends won't buy this analysis, let alone the next one we propose.",
 'label': 1,
 'idx': 0}

In [6]:
def f(examples, tokenizer):
    return tokenizer.batch_encode_plus(examples['sentence'], truncation=True)

dataset = dataset.map(f,
                     batched=True,
                     batch_size=1000,
                      # num_proc=1更快, 数据量不多的时候, 创建进程也是需要时间开销的.
                     num_proc=1,
                     remove_columns=['sentence', 'idx'],
                     fn_kwargs={'tokenizer': tokenizer})



  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [8]:
print(dataset['train'][0])

{'label': 1, 'input_ids': [101, 2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [9]:
import torch
from transformers.data.data_collator import DataCollatorWithPadding

In [10]:
loader = torch.utils.data.DataLoader(
    dataset=dataset['train'],
    batch_size=8,
    collate_fn=DataCollatorWithPadding(tokenizer),
    shuffle=True,
    drop_last=True,
)

for data in loader:
    break

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [11]:
data

{'input_ids': tensor([[  101,  1996,  2062,  4180,  2098,  2057, 14688,  2000,  2022,  1010,
          1996,  2062,  2057,  3473,  4854,  2012,  1996,  7435,  1012,   102],
        [  101,  2040, 17749,  2073,  2057,  4149,  2054,  1029,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101, 18500,  2245,  2008,  2002,  2001,  1996,  3159,  1997,  4768,
          1012,   102,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  1999,  1996,  4020,  2045, 28374,  1037,  2543,  1012,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  2198, 14140,  2070,  2769,  1999,  1996,  2924,  2006,  5958,
          1012,   102,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  2320,  9965,  2187,  1010,  5965,  2150,  2035,  1996, 13675,
         16103,  2121,  1012,   102,     0,     0,     0,     0,     0,     0],
        [  101,  2023,  2311,  2

In [12]:
len(loader)

1068

In [13]:
from transformers import AutoModelForSequenceClassification, DistilBertModel

In [14]:
parameters = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
parameters

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [45]:
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.pretrained = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.fc = torch.nn.Sequential(torch.nn.Linear(768, 768),
                                      torch.nn.ReLU(),
                                      torch.nn.Dropout(p=0.2),
                                      torch.nn.Linear(768, 2)
                                     )
        
        parameters = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
        self.fc[0].load_state_dict(parameters.pre_classifier.state_dict())
        self.fc[3].load_state_dict(parameters.classifier.state_dict())
        
        self.criterion = torch.nn.CrossEntropyLoss()
        
    def forward(self, input_ids, attention_mask, labels=None):
        logits = self.pretrained(input_ids=input_ids, attention_mask=attention_mask)
        logits = logits.last_hidden_state[:, 0]
        logits = self.fc(logits)
        
        loss = None
        if labels is not None:
#             print('logits: ', logits)
#             print('labels: ', labels)
            loss = self.criterion(logits, labels)
            
        return {'loss': loss, 'logits': logits}

In [46]:
model = Model()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_lay

In [17]:
print(sum(i.numel() for i in model.parameters()))

66955010


In [47]:
out = model(**data)
out['loss'], out['logits'].shape

(tensor(0.6949, grad_fn=<NllLossBackward0>), torch.Size([8, 2]))

In [23]:
data

{'input_ids': tensor([[  101,  1996,  2062,  4180,  2098,  2057, 14688,  2000,  2022,  1010,
          1996,  2062,  2057,  3473,  4854,  2012,  1996,  7435,  1012,   102],
        [  101,  2040, 17749,  2073,  2057,  4149,  2054,  1029,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101, 18500,  2245,  2008,  2002,  2001,  1996,  3159,  1997,  4768,
          1012,   102,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  1999,  1996,  4020,  2045, 28374,  1037,  2543,  1012,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  2198, 14140,  2070,  2769,  1999,  1996,  2924,  2006,  5958,
          1012,   102,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  2320,  9965,  2187,  1010,  5965,  2150,  2035,  1996, 13675,
         16103,  2121,  1012,   102,     0,     0,     0,     0,     0,     0],
        [  101,  2023,  2311,  2

In [35]:
dataset['test'][0]

{'label': -1,
 'input_ids': [101, 3021, 26265, 2627, 1996, 2160, 1012, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [36]:
dataset['validation'][0]

{'label': 1,
 'input_ids': [101,
  1996,
  11279,
  8469,
  1996,
  9478,
  3154,
  1997,
  1996,
  5749,
  1012,
  102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [48]:
# 测试
def test(model):
    model.eval()
    
    loader_test = torch.utils.data.DataLoader(
        dataset=dataset['validation'],
        batch_size=16,
        collate_fn=DataCollatorWithPadding(tokenizer),
        shuffle=True,
        drop_last=True
    )
    
    outs = []
    labels = []
    for i, data in enumerate(loader_test):
#         print(data)
        with torch.no_grad():
            out = model(**data)
            
        outs.append(out['logits'].argmax(dim=1))
        labels.append(data['labels'])
        
        
        if i % 10 == 0:
            print(i)
            
        if i == 50:
            break
            
    outs = torch.cat(outs)
    labels = torch.cat(labels)
#     print('test labels:', labels)
    accuracy = (outs == labels).sum().item() / len(labels)
    print('accuracy: ', accuracy)

In [49]:
test(model)

0
10
20
30
40
50
accuracy:  0.5502450980392157


In [50]:
from transformers import AdamW
from transformers.optimization import get_scheduler

In [51]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [54]:

# 训练代码
def train():
    
    # 定义优化器
    optimizer = AdamW(model.parameters(), betas=(0.9, 0.999), eps=1e-8, lr=2e-5)
    
    scheduler = get_scheduler(name='linear',
                             num_warmup_steps=0,
                             num_training_steps=len(loader),
                             optimizer=optimizer)
    
    model.to(device)
    model.train()
    
    for i, data in enumerate(loader):
        input_ids, attention_mask, labels  = data['input_ids'], data['attention_mask'], data['labels']
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        
        out = model(input_ids=input_ids, 
                    attention_mask=attention_mask, 
                    labels=labels)
        loss = out['loss']
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        optimizer.zero_grad()
        model.zero_grad()
        
        if i % 50 == 0:
            lr = optimizer.state_dict()['param_groups'][0]['lr']
            out = out['logits'].argmax(dim=1)
            
            accuracy = (labels==out).sum().item() / 8
            
            print(i, loss.item(), lr, accuracy)
            print()

In [55]:
train()

D:\.venv\lib\site-packages\transformers\optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


0 0.6962183713912964 1.9981273408239703e-05 0.5

50 0.377118319272995 1.9044943820224723e-05 0.875

100 0.5701767206192017 1.810861423220974e-05 0.625

150 0.3946194648742676 1.7172284644194758e-05 1.0

200 0.5135958790779114 1.6235955056179777e-05 0.875

250 0.8008895516395569 1.5299625468164797e-05 0.5

300 0.7711583375930786 1.4363295880149814e-05 0.625

350 0.5669834017753601 1.3426966292134834e-05 0.75

400 0.4095596373081207 1.2490636704119851e-05 0.875

450 0.4815349578857422 1.155430711610487e-05 0.75

500 0.3883439302444458 1.0617977528089888e-05 0.75

550 0.23434026539325714 9.681647940074908e-06 0.875

600 0.31013554334640503 8.745318352059925e-06 0.875

650 0.3703773021697998 7.808988764044945e-06 0.875

700 0.23676088452339172 6.872659176029963e-06 0.875

750 0.4773671627044678 5.936329588014982e-06 0.875

800 0.4274585247039795 5e-06 0.75

850 0.6012129187583923 4.063670411985019e-06 0.875

900 0.20150703191757202 3.1273408239700374e-06 1.0

950 0.45682013034820557 2.1910

In [57]:
test(model.to('cpu'))

0
10
20
30
40
50
accuracy:  0.7781862745098039
